<a href="https://colab.research.google.com/github/Chinh-tran204/solfDrinkRegconitionUsingCNN/blob/main/solfDrinkReg_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kết nối với drive để trích xuất file data
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#nhúng các thư viện cần thiết chi việc học ảnh
from os import listdir
from numpy import asarray
from numpy import save
from keras.utils.image_utils import img_to_array
from keras.utils import load_img

In [ ]:
#Chọn vào thư mục có chứa data để train
folder = '/content/drive/MyDrive/datasets/data_solf_drink/'
photos, labels = list(), list()

#Gán nhãn cho các vật được đem train cho AI
for file in listdir(folder):
  output = 0.0
  if file.startswith('pepsi'):
    output = 1.0  #'pepsi'
  if file.startswith('coca'):
    output = 2.0  #'coca'
  if file.startswith('7up'):
    output = 3.0  #'7up'
  if file.startswith('mirinda'):
    output = 4.0  #'mirinda'
  if file.startswith('revive'):
    output = 5.0  #'revive'
  if file.startswith('sting'):
    output = 6.0  #'sting'

  #Thêm các dữ liệu vừa đọc vào một file photo và một file labels tạm để tiện cho việc train
  photo = load_img (folder+file, target_size= (256,256))
  photo = img_to_array (photo)
  photos.append(photo)
  labels.append(output)

#Chuyển dạng data về array và lưu lại trong drive
photos = asarray(photos)
labels = asarray(labels)
save('/content/drive/MyDrive/datasets/solfDrink_photo.npy', photos)
save('/content/drive/MyDrive/datasets/solfDrink_label.npy', labels)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#load lại giá trị mới 
photo = np.load('/content/drive/MyDrive/datasets/solfDrink_photo.npy')
label = np.load('/content/drive/MyDrive/datasets/solfDrink_label.npy')

In [ ]:
#Tách data cho việc train và test
split_index = int(0.2*len(photo))
test_X,test_Y = photo[:split_index],label[:split_index]
train_X, train_Y = photo[split_index:],label[split_index:]

In [ ]:
print('Train:',train_X.shape)
print('Test:',test_X.shape)

Train: (145, 256, 256, 3)
Test: (36, 256, 256, 3)


In [ ]:
#Reshape lại đầu vào cho AI
train_X = train_X.reshape((145,256,256,3))
train_X = train_X.astype('float32')/255
test_X = test_X.reshape((36,256,256,3))
test_X = test_X.astype('float32')/255

In [ ]:
from keras.utils import to_categorical
train_Y = to_categorical(train_Y,7)
test_Y = to_categorical(test_Y,7)

In [ ]:
#Khai báo các thư viện cần thiết cho việc train AI
from keras.models import Sequential
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Normalization
from keras.optimizers import Adam
from keras.layers import LeakyReLU

In [ ]:
#Xác định các thông số cho việc học của AI
batch_size = 52
epochs = 100
classes = 7

In [ ]:
#Tạo các lớp nơ ron cần thiết cho việc train AI
model = Sequential()
model.add(Conv2D(128, kernel_size= (3,3), activation= 'linear', input_shape= (256,256,3), padding= 'same'))
model.add(LeakyReLU(alpha= 0.1))
model.add(MaxPooling2D((2,2), padding= 'same'))

model.add(Conv2D(128,(3,3), activation= 'linear', padding= 'same'))
model.add(LeakyReLU(alpha= 0.1))
model.add(MaxPooling2D((2,2), padding= 'same'))

model.add(Conv2D(256,(3,3), activation= 'linear', padding= 'same'))
model.add(LeakyReLU(alpha= 0.1))
model.add(MaxPooling2D((2,2), padding= 'same'))

model.add(Flatten())
model.add(Dense(256, activation= 'linear'))
model.add(LeakyReLU(alpha= 0.1))
model.add(Dense(classes, activation= 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 128)     3584      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256, 256, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     147584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128, 128, 128)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                    

In [ ]:
#Load model đã tạo và bắt đầu cho train với những thông số đã cài trước đó
from keras.losses import categorical_crossentropy
model.compile(optimizer= Adam(), loss= categorical_crossentropy, metrics= ['accuracy'])
train = model.fit(train_X, train_Y, epochs= epochs, batch_size= batch_size, verbose= 1)

Epoch 1/100
3/3 [==============================] - 22s 3s/step - loss: 27.0523 - accuracy: 0.1517
Epoch 2/100
3/3 [==============================] - 1s 325ms/step - loss: 26.3962 - accuracy: 0.1172
Epoch 3/100
3/3 [==============================] - 1s 327ms/step - loss: 17.4554 - accuracy: 0.2069
Epoch 4/100
3/3 [==============================] - 1s 333ms/step - loss: 4.2997 - accuracy: 0.1517
Epoch 5/100
3/3 [==============================] - 1s 315ms/step - loss: 2.7489 - accuracy: 0.1655
Epoch 6/100
3/3 [==============================] - 1s 314ms/step - loss: 1.6890 - accuracy: 0.3034
Epoch 7/100
3/3 [==============================] - 1s 315ms/step - loss: 1.6637 - accuracy: 0.2621
Epoch 8/100
3/3 [==============================] - 1s 318ms/step - loss: 1.4947 - accuracy: 0.3655
Epoch 9/100
3/3 [==============================] - 1s 317ms/step - loss: 1.3240 - accuracy: 0.4690
Epoch 10/100
3/3 [==============================] - 1s 315ms/step - loss: 1.0949 - accuracy: 0.5793
Epoch 11

In [ ]:
ST#Lưu lại model để dễ cho việc làm việc
model.save('/content/drive/MyDrive/datasets/solfDrinkReg_cnn.h5')